<table width = "100%">
  <tr style="background-color:white;">
    <!-- QWorld Logo -->
    <td style="text-align:left;width:200px;"> 
        <a href="https://qworld.net/" target="_blank"><img src="../images/QWorld.png"> </a></td>
    <td style="text-align:right;vertical-align:bottom;font-size:16px;"> 
        Prepared by Özlem Salehi</td>    
</table>
<hr>

# <font color="blue"> Solutions for </font> Quantum Annealing on D-Wave


### Imports

In [24]:
import dimod
from dimod import BQM
import networkx as nx
import numpy as np
from dwave.system import EmbeddingComposite
from dwave.system import DWaveSampler

from bqm_utils import graph_viz, tsp_viz

<a id="Task1"></a>

### Task 1

Solve TSP problem for the following graph with 8 vertices. Try different number of reads and interpret the results. 

Note: Don't forget to set a proper penalty coefficient.

In [25]:
np.random.seed(45)
N = 8
G2 = nx.complete_graph(N)
for u, v in G2.edges():
    G2[u][v]["weight"] = np.random.randint(1, 10)

### Solution

In [26]:
def tsp_bqm(G, P):
    
    N = len(G.nodes)
    bqm = BQM("BINARY")
    for i in range(N):
        for j in range(N):
            if i!=j:
                for t in range(N-1):
                    bqm.add_quadratic(f"x_{i}_{t}", f"x_{j}_{t+1}", G[i][j]["weight"])

                #Remember that we were assuming N=0 in the sum
                bqm.add_quadratic(f"x_{i}_{N-1}", f"x_{j}_{0}", G[i][j]["weight"])
    #Add the first constraint
    for t in range(N):
        c1 = [(f"x_{i}_{t}", 1) for i in range(N)] #coefficient list
        bqm.add_linear_equality_constraint(c1, constant=-1, lagrange_multiplier=P)
    #Add the second constraint
    for i in range(N):
        c2 = [(f"x_{i}_{t}", 1) for t in range(N)]
        bqm.add_linear_equality_constraint(c2, constant=-1, lagrange_multiplier=P)
    return bqm

def is_sample_feasible(sample,N):
    for i in range(N):
        if sum(sample[f"x_{i}_{t}"] for t in range(N))!=1:
            return False
    for t in range(N):
        if sum(sample[f"x_{i}_{t}"] for i in range(N))!=1:
            return False
    return True

def sample_to_path(sample,N):
    path = []
    for t in range(N):
        for i in range(N):
            if sample[f"x_{i}_{t}"] == 1:
                path.append(i)
    return path

In [27]:
#We take penalty coefficient as 10
bqm_2 = tsp_bqm(G2, 10)
sampler = EmbeddingComposite(DWaveSampler())

In [24]:
sampleset = sampler.sample(bqm_2, num_reads=1000)

In [25]:
for s in sampleset.data():
    print(is_sample_feasible(s.sample,N))
    print(sample_to_path(s.sample,N),s.energy)

False
[5, 4, 6, 3, 0, 1, 2] 47.0
False
[4, 0, 2, 5, 7, 3, 6, 1] 50.0
False
[3, 4, 5, 7, 2, 0, 0, 1] 50.0
False
[3, 2, 5, 7, 6, 1, 4] 52.0
False
[5, 4, 0, 3, 7, 2] 53.0
False
[3, 3, 2, 4, 5, 0, 7, 1] 54.0
False
[6, 7, 3, 2, 5, 0, 1] 54.0
False
[7, 2, 5, 4, 1, 3] 54.0
False
[2, 0, 3, 5, 4, 6] 56.0
False
[6, 5, 2, 5, 7, 1, 0, 4] 56.0
False
[2, 5, 3, 1, 0, 7] 56.0
False
[0, 4, 5, 3, 1, 2, 6, 7] 56.0
False
[6, 4, 5, 7, 2, 0] 56.0
False
[3, 3, 0, 4, 7, 5, 2] 57.0
False
[5, 2, 1, 3, 4, 0, 7, 2] 57.0
False
[7, 0, 6, 4, 2, 3, 2, 5] 57.0
False
[6, 0, 4, 3, 2, 1, 5, 7] 59.0
False
[1, 3, 5, 4, 6, 2, 2, 7] 59.0
False
[7, 5, 2, 4, 3, 1] 59.0
False
[1, 2, 4, 5, 3, 0, 7, 6] 60.0
False
[6, 4, 5, 7, 3, 1, 0, 2] 60.0
False
[5, 3, 4, 4, 7, 0, 2, 1] 60.0
False
[1, 3, 4, 5, 7, 0, 2, 6] 60.0
False
[4, 5, 3, 0, 7, 2] 60.0
False
[6, 4, 0, 7, 5, 2] 60.0
False
[6, 1, 2, 4, 3, 7, 0] 60.0
False
[1, 0, 7, 3, 2, 5, 2] 61.0
False
[2, 3, 6, 4, 5, 7, 0, 1] 61.0
False
[7, 1, 2, 0, 3, 5] 61.0
False
[2, 1, 7, 3, 3, 4, 5] 

False
[1, 6, 7, 4, 0, 2, 7, 5] 87.0
False
[6, 1, 5, 4, 0, 2, 5, 7, 3] 87.0
False
[1, 5, 3, 2, 4, 0, 4] 87.0
False
[3, 6, 7, 3, 2, 5, 2] 87.0
False
[7, 4, 5, 7, 0, 5, 2, 3] 87.0
False
[3, 7, 1, 5, 2, 5, 0] 88.0
False
[1, 5, 0, 4, 3, 2, 7, 5] 88.0
False
[0, 1, 3, 2, 5, 4, 7, 6] 88.0
False
[6, 7, 0, 1, 3, 7, 0, 5, 4] 88.0
False
[6, 5, 5, 0, 3, 3, 1, 2, 7] 88.0
False
[4, 5, 7, 0, 1, 7] 88.0
False
[0, 1, 5, 2, 6] 88.0
False
[5, 5, 0, 4, 2, 0, 3, 7, 1] 88.0
False
[7, 4, 2, 5, 6, 7, 0, 1] 88.0
False
[0, 7, 1, 2, 5, 3] 88.0
False
[6, 0, 2, 4, 3, 3] 88.0
False
[2, 4, 5, 1, 7, 5, 3, 6] 88.0
False
[7, 2, 0, 2, 5, 7, 1, 3] 88.0
False
[0, 3, 7, 5, 6, 3, 1, 2] 88.0
False
[1, 6, 5, 4, 7, 0, 3, 5, 2] 88.0
False
[1, 7, 5, 0, 2, 3, 5] 88.0
False
[0, 1, 1, 6, 7, 2, 0, 5] 88.0
False
[3, 0, 2, 7, 6, 5] 88.0
False
[4, 3, 2, 5, 7, 1] 88.0
False
[0, 3, 7, 1, 6, 4, 5, 1, 2] 88.0
False
[1, 0, 2, 3, 4, 6, 6, 4, 5] 88.0
False
[1, 0, 5, 3, 7, 2, 2, 6, 4] 89.0
False
[5, 7, 2, 3, 0, 6, 7] 89.0
False
[1, 0, 2, 4, 3, 

[6, 1, 3, 0, 7, 2, 5, 5, 2] 104.0
False
[0, 3, 5, 7, 5, 1, 2] 105.0
False
[0, 4, 7, 1, 2, 3, 0, 1, 5] 105.0
False
[5, 5, 3, 4, 0, 3, 7] 105.0
False
[5, 5, 3, 2, 0, 1, 1, 2] 105.0
False
[5, 3, 4, 5, 2, 6, 0, 1, 0] 105.0
False
[5, 4, 5, 7, 3, 6, 2, 1] 105.0
False
[3, 4, 6, 5, 7, 0, 3, 2, 7] 105.0
False
[3, 1, 7, 2, 6, 2, 4, 0, 5, 0] 105.0
False
[0, 7, 4, 6, 3, 3, 7, 1] 105.0
False
[5, 1, 2, 6, 2, 0, 2, 3] 105.0
False
[7, 0, 1, 4, 6, 3, 5, 7, 5] 105.0
False
[0, 2, 3, 7, 1, 4, 0, 5, 2] 105.0
False
[0, 2, 6, 6, 7, 5, 1, 2] 105.0
False
[4, 4, 6, 5, 2, 0, 3, 5, 1] 105.0
False
[4, 7, 1, 0, 3, 3, 6, 2] 105.0
False
[3, 0, 4, 0, 7, 2, 5, 1, 2] 106.0
False
[0, 7, 2, 5, 0, 2, 4, 1, 3] 106.0
False
[1, 0, 7, 2, 5, 0, 7] 106.0
False
[0, 3, 1, 2, 4, 7, 5, 5, 4] 106.0
False
[6, 2, 1, 3, 0, 0, 5, 7] 106.0
False
[1, 0, 0, 3, 7, 1, 2] 106.0
False
[3, 4, 1, 2, 6, 2, 5, 1] 106.0
False
[5, 0, 7, 1, 3, 6, 2, 3, 6] 106.0
False
[3, 4, 2, 6, 0, 2, 5, 0] 106.0
False
[3, 6, 0, 4, 4, 1, 5, 2] 107.0
False
[0, 3, 5, 3

[1, 7, 5, 6, 7, 0, 0, 3, 2, 4] 132.0
False
[7, 7, 0, 5, 2, 3, 5, 6, 2, 4] 132.0
False
[6, 0, 4, 0, 1, 7, 2, 5] 133.0
False
[2, 1, 1, 5, 3, 1, 0, 5, 4] 133.0
False
[2, 1, 2, 3, 0, 2] 133.0
False
[3, 1, 5, 7, 2, 6, 0, 5, 7] 133.0
False
[1, 7, 0, 6, 0, 1, 3, 4] 133.0
False
[1, 2, 3, 6, 4, 0, 6, 0, 3] 133.0
False
[3, 1, 5, 7, 2, 2, 6, 6, 4, 5, 0] 133.0
False
[6, 7, 4, 3, 5, 1, 3, 1] 133.0
False
[1, 6, 2, 3, 4, 7, 5, 1, 2, 7] 134.0
False
[6, 7, 0, 3, 5, 1, 5, 6, 2] 134.0
False
[1, 7, 5, 0, 6, 5, 7, 4] 134.0
False
[3, 0, 6, 4, 6, 4, 5, 5] 134.0
False
[0, 4, 0, 1, 5, 1, 2, 3, 5, 7] 134.0
False
[4, 6, 4, 7, 0, 7, 1, 2] 134.0
False
[0, 3, 5, 5, 7, 5, 2] 134.0
False
[3, 6, 0, 2, 5, 1, 2, 1, 6] 134.0
False
[1, 0, 0, 3, 6, 2, 4, 1, 2] 134.0
False
[0, 5, 2, 1, 4, 3, 2, 3, 4] 134.0
False
[4, 1, 4, 5, 0, 7, 5, 3, 5, 2] 134.0
False
[3, 0, 5, 4, 4, 2, 5, 0] 134.0
False
[7, 1, 2, 5, 5, 1, 3, 7] 135.0
False
[2, 3, 5, 7, 5, 0, 5, 1, 2, 6] 135.0
False
[3, 5, 6, 0, 6, 1, 2, 7] 135.0
False
[5, 7, 4, 7, 6, 3,

For 1000 reads, we don't have any feasible solution. Let's try increasing the number of reads to 2000.

In [26]:
sampleset2 = sampler.sample(bqm_2, num_reads=2000)

In [27]:
for s in sampleset2.data():
    print(is_sample_feasible(s.sample,N))
    print(sample_to_path(s.sample,N),s.energy)

True
[0, 4, 5, 7, 2, 6, 3, 1] 38.0
True
[3, 2, 1, 7, 5, 4, 6, 0] 41.0
False
[3, 2, 1, 5, 4, 0, 7] 42.0
False
[3, 2, 5, 4, 1, 0, 7] 42.0
True
[3, 7, 1, 0, 6, 2, 5, 4] 44.0
True
[0, 2, 7, 5, 6, 4, 3, 1] 44.0
True
[3, 1, 0, 4, 7, 5, 6, 2] 45.0
False
[1, 2, 0, 7, 3, 4, 5] 45.0
False
[3, 7, 0, 1, 6, 5, 2] 45.0
True
[1, 7, 0, 6, 4, 3, 5, 2] 46.0
False
[2, 0, 1, 6, 4, 5, 3] 47.0
False
[4, 0, 2, 3, 7, 6, 1] 49.0
False
[7, 1, 0, 6, 5, 2, 3] 49.0
False
[1, 0, 7, 5, 3, 4, 6] 49.0
False
[4, 1, 7, 6, 2, 3, 5] 50.0
False
[3, 2, 6, 7, 0, 4, 5, 3] 50.0
False
[3, 6, 0, 1, 2, 5, 7] 50.0
False
[0, 6, 5, 4, 3, 2, 7] 50.0
False
[4, 5, 3, 2, 0, 1] 51.0
True
[3, 2, 4, 1, 6, 5, 0, 7] 51.0
False
[1, 7, 3, 5, 2, 4, 0] 51.0
False
[3, 1, 0, 6, 2, 5, 4] 51.0
False
[3, 7, 4, 5, 1, 2, 0] 51.0
False
[3, 1, 0, 2, 6, 7, 5] 51.0
False
[6, 7, 0, 1, 5, 2, 3] 51.0
False
[3, 7, 5, 5, 6, 4, 0, 2] 51.0
False
[6, 2, 0, 7, 3, 5, 1] 52.0
False
[6, 7, 3, 5, 2, 0, 1] 52.0
False
[3, 1, 0, 6, 7, 5, 2] 52.0
False
[0, 0, 2, 3, 7, 5, 6

False
[2, 0, 0, 7, 1, 5, 1, 3] 75.0
False
[1, 3, 7, 2, 5, 0, 4, 3] 75.0
False
[6, 7, 0, 3, 1, 0, 4, 5, 2] 75.0
False
[0, 4, 4, 2, 1, 3, 7, 5] 75.0
False
[0, 3, 5, 2, 5, 4, 7, 1] 75.0
False
[4, 0, 2, 1, 5] 75.0
False
[6, 7, 2, 0, 3, 1, 4] 75.0
False
[7, 4, 4, 3, 2, 0, 5] 75.0
False
[2, 1, 0, 7, 3, 6, 5] 76.0
False
[7, 3, 1, 0, 4, 6, 4] 76.0
False
[0, 1, 7, 5, 4, 6, 2, 7] 76.0
False
[7, 1, 2, 5, 4, 3, 6] 76.0
False
[5, 5, 0, 7, 1, 3, 2, 4] 76.0
False
[3, 5, 0, 2, 7, 4, 6, 2] 76.0
False
[4, 7, 0, 2, 1, 0] 76.0
False
[3, 1, 7, 5, 0, 5, 2] 76.0
False
[3, 1, 0, 4, 7, 5, 2, 1, 3] 76.0
False
[1, 3, 2, 6, 7, 5] 76.0
False
[3, 1, 6, 0, 4, 5, 1, 2] 76.0
False
[7, 0, 5, 7, 6, 2, 3] 76.0
False
[2, 6, 1, 0, 3, 4, 5] 76.0
False
[0, 5, 1, 7, 4, 3, 7] 76.0
False
[2, 1, 4, 2, 7, 5, 3, 0] 76.0
False
[6, 7, 0, 5, 0, 2, 2, 3, 4] 76.0
False
[2, 5, 6, 7, 7, 1, 2, 3] 76.0
False
[1, 0, 2, 7, 3, 5, 7] 76.0
False
[3, 6, 7, 0, 2, 5, 7] 76.0
False
[3, 4, 6, 5, 5, 2, 0] 77.0
False
[0, 2, 1, 5, 6, 3, 7] 77.0
False
[

[3, 6, 1, 0, 7, 1, 5] 86.0
False
[3, 2, 6, 0, 1, 4, 2, 5] 86.0
False
[1, 5, 4, 6, 0, 3, 2, 0, 1] 86.0
False
[1, 1, 2, 7, 6, 0, 5] 86.0
False
[7, 7, 0, 2, 6, 5, 1, 4] 86.0
False
[1, 5, 7, 2, 3, 0, 2, 6] 87.0
False
[3, 4, 0, 1, 2, 5, 0, 2] 87.0
False
[0, 3, 0, 5, 7, 1, 6] 87.0
False
[3, 5, 2, 0, 6, 0, 7, 3] 87.0
False
[0, 3, 7, 7, 0, 1, 5, 4, 6, 2] 87.0
False
[3, 1, 2, 5, 5, 4, 6, 7] 87.0
False
[2, 6, 0, 3, 2, 5, 1, 7] 87.0
False
[0, 7, 1, 2, 5, 3, 4] 87.0
False
[6, 7, 0, 3, 2, 2, 4, 5] 87.0
False
[5, 2, 0, 4, 0] 87.0
False
[3, 7, 0, 5, 5, 6, 2, 4] 87.0
False
[5, 3, 1, 2, 5, 7] 87.0
False
[7, 5, 4, 0, 0, 3, 3, 2, 5] 87.0
False
[1, 1, 3, 4, 2, 5, 6] 87.0
False
[3, 7, 6, 0, 0, 1, 2, 5] 87.0
False
[1, 1, 5, 0, 3] 87.0
False
[3, 5, 2, 2, 7, 1, 4, 5] 87.0
False
[7, 0, 1, 4, 5, 2, 3, 0] 87.0
False
[0, 2, 5, 5, 1, 0, 7, 3] 87.0
False
[2, 3, 5, 4, 1, 0, 6, 7, 6] 87.0
False
[3, 1, 0, 1, 4, 6, 7, 5] 87.0
False
[1, 2, 4, 0, 3, 5, 0, 6, 7] 87.0
False
[0, 5, 6, 7, 1, 1, 2, 3] 87.0
False
[7, 3, 6, 4, 

[7, 5, 0, 2, 2, 1, 1] 96.0
False
[1, 7, 0, 6, 1, 3, 5, 2, 4] 96.0
False
[2, 1, 7, 0, 6, 5, 7, 2] 96.0
False
[2, 3, 7, 1, 7, 0, 6, 4, 5] 96.0
False
[2, 7, 0, 1, 4, 5, 6, 3] 96.0
False
[5, 5, 0, 3, 3, 4, 6] 96.0
False
[7, 6, 0, 3, 5, 1, 2] 96.0
False
[5, 2, 0, 5, 3, 5, 4, 6] 96.0
False
[3, 2, 0, 7, 1, 1, 2, 6] 96.0
False
[4, 1, 6, 7, 5, 5, 7, 2] 96.0
False
[2, 7, 0, 5, 2, 4] 96.0
False
[2, 1, 7, 6, 3, 0, 7] 96.0
False
[3, 0, 6, 1, 7, 0, 4, 7, 5] 96.0
False
[0, 2, 0, 5, 5, 4, 3] 96.0
False
[2, 4, 7, 7, 0, 5, 6, 1, 3] 96.0
False
[1, 0, 2, 3, 5, 6, 0] 96.0
False
[7, 6, 0, 5, 1, 4, 5] 96.0
False
[6, 5, 7, 3, 5, 1, 2] 97.0
False
[3, 0, 5, 1, 5, 3, 7] 97.0
False
[2, 6, 7, 7, 1, 0, 5, 3, 4] 97.0
False
[4, 2, 5, 7, 3, 1, 6, 0, 7] 97.0
False
[4, 6, 0, 5, 6, 2, 7] 97.0
False
[1, 4, 3, 0, 7, 1] 97.0
False
[1, 5, 7, 3, 7, 3, 6, 2, 0, 4] 97.0
False
[1, 6, 3, 0, 5, 6, 2, 2] 97.0
False
[5, 7, 4, 7, 0, 0, 1, 2] 97.0
False
[3, 7, 5, 0, 3, 6, 5, 2] 97.0
False
[0, 5, 0, 1, 7, 2, 4, 3, 1] 97.0
False
[2, 0, 

[7, 2, 0, 5, 2, 5, 1, 0] 105.0
False
[4, 2, 5, 6, 1, 2, 0, 3, 7, 3] 105.0
False
[7, 4, 0, 6, 1, 2, 5, 5, 0] 105.0
False
[2, 0, 0, 4, 7, 5, 5, 3, 7] 105.0
False
[7, 3, 6, 7, 5, 0, 1, 2, 1] 105.0
False
[3, 4, 5, 6, 1, 7, 0, 0, 2, 3] 105.0
False
[3, 7, 0, 5, 7, 2, 1, 3] 105.0
False
[3, 0, 6, 7, 5, 6, 2, 2] 105.0
False
[3, 0, 4, 5, 0, 1, 7] 105.0
False
[2, 7, 1, 2, 5, 0, 3] 105.0
False
[1, 3, 2, 7, 5, 5, 0, 0, 4, 1] 105.0
False
[1, 0, 7, 3, 5, 5, 7, 2] 105.0
False
[6, 7, 0, 0, 2, 5, 5, 2] 105.0
False
[0, 2, 2, 1, 6, 5, 4, 7, 1] 105.0
False
[1, 0, 7, 0, 3, 5, 2, 5] 105.0
False
[3, 5, 1, 4, 2, 7, 6, 3, 7] 106.0
False
[7, 6, 4, 0, 6, 7, 5, 2, 5] 106.0
False
[7, 5, 2, 0, 3, 6, 5, 7, 2] 106.0
False
[0, 4, 1, 6, 3, 5, 0, 2, 5, 7] 106.0
False
[0, 3, 4, 0, 5, 7, 2, 5] 106.0
False
[0, 1, 2, 4, 7, 3, 5, 5, 6, 7] 106.0
False
[0, 6, 4, 7, 5, 3, 5, 2, 3, 2] 106.0
False
[3, 7, 5, 0, 4, 5, 5, 2, 3] 106.0
False
[5, 1, 4, 6, 5, 0, 3, 7, 7] 106.0
False
[3, 4, 0, 2, 3, 0, 5, 7] 106.0
False
[5, 2, 4, 0, 5, 7,

[0, 1, 3, 0, 2, 3] 117.0
False
[2, 5, 0, 4, 1, 7, 6, 7] 117.0
False
[5, 7, 1, 3, 6, 6, 7, 2] 117.0
False
[3, 1, 6, 0, 7, 3, 4, 5] 117.0
False
[7, 5, 0, 0, 1, 4, 6, 3] 117.0
False
[2, 5, 0, 0, 2, 7, 6, 2] 117.0
False
[2, 0, 5, 6, 4, 4, 3, 7] 117.0
False
[1, 2, 2, 5, 1, 6, 4, 3] 117.0
False
[2, 3, 7, 5, 1, 4, 1, 1, 6] 117.0
False
[1, 7, 2, 0, 7, 5, 6, 4, 6] 117.0
False
[3, 7, 2, 0, 7, 6, 4, 5, 1] 117.0
False
[2, 0, 1, 6, 2, 5, 5, 4, 7] 117.0
False
[4, 7, 1, 5, 6, 7, 2] 117.0
False
[4, 1, 0, 1, 7, 7, 6, 5, 6] 117.0
False
[4, 0, 4, 1, 3, 1, 5, 2, 7, 7] 117.0
False
[1, 1, 4, 1, 3, 5, 7] 117.0
False
[5, 2, 7, 1, 5, 4, 3, 2, 7] 118.0
False
[1, 6, 7, 5, 4, 6, 5] 118.0
False
[3, 6, 7, 0, 7, 1, 4, 2, 4] 118.0
False
[0, 2, 7, 0, 1, 7, 6] 118.0
False
[6, 7, 1, 4, 0, 3, 0, 5, 3] 118.0
False
[4, 2, 7, 1, 2, 0, 7, 6, 3] 118.0
False
[2, 3, 1, 7, 2, 2, 6] 118.0
False
[3, 7, 0, 0, 7, 6, 4, 4, 5, 1] 118.0
False
[0, 1, 0, 2, 5, 5, 4, 6] 118.0
False
[3, 2, 0, 1, 0, 1, 7, 2, 5, 5] 118.0
False
[2, 5, 7, 3, 5

False
[4, 7, 5, 6, 6, 5, 3, 4] 132.0
False
[1, 2, 4, 0, 1, 2, 5, 5, 7, 6] 132.0
False
[0, 6, 2, 7, 5, 6, 1, 2] 132.0
False
[5, 4, 0, 6, 4, 5, 7, 2, 3, 6] 133.0
False
[2, 3, 2, 0, 1, 0, 6, 4, 5, 7, 7] 133.0
False
[5, 0, 1, 2, 6, 1, 2] 133.0
False
[2, 2, 6, 0, 2, 0, 3, 5] 133.0
False
[1, 6, 1, 7, 4, 4, 5, 0, 2, 5] 133.0
False
[2, 7, 0, 5, 6, 2, 4, 5, 3, 4] 133.0
False
[0, 1, 6, 0, 2, 2, 3, 3] 133.0
False
[0, 1, 6, 7, 3, 3, 0, 5] 133.0
False
[6, 5, 1, 7, 0, 6, 0, 2, 5] 133.0
False
[5, 1, 0, 1, 0, 2, 7, 0, 3, 5] 133.0
False
[0, 3, 0, 6, 1, 4, 7, 2, 2, 5] 133.0
False
[0, 2, 7, 1, 4, 5, 1, 6, 3] 133.0
False
[3, 7, 5, 0, 1, 2, 6, 7, 5] 133.0
False
[3, 7, 0, 1, 2, 2, 3, 3, 4] 133.0
False
[3, 3, 2, 5, 7, 1, 3, 7] 134.0
False
[1, 2, 0, 2, 5, 7, 5, 5, 4, 6] 134.0
False
[4, 1, 1, 5, 4, 0, 3, 5, 7] 134.0
False
[3, 4, 5, 0, 2, 5, 7, 2, 3] 134.0
False
[1, 7, 2, 5, 3, 3, 0, 1] 134.0
False
[2, 1, 0, 4, 5, 6, 2, 1, 3, 6] 134.0
False
[1, 0, 5, 4, 4, 5, 6] 134.0
False
[5, 0, 1, 1, 5, 4, 7, 7] 134.0
False


This time, we have some feasible solutions. The first one has the lowest energy, yet we don't know if it is optimal or not and we can not know this unless we use an exact solver.

<a id="Task2"></a>

### Task 2

For the graph G2, use quantum annealing with `annealing_time` set to 100, `num_reads` set to 2000 and relative chain strength set to 0.1, 0.2, 0.3, 0.4, 0.5 and evaluate the results by getting whether the sample is feasible, the path, energy and fraction of broken chains.

### Solution

In [28]:
chain_strengths = [0.1, 0.2, 0.3, 0.4, 0.5]
max_val = max([abs(x) for x in bqm_2.linear.values()] + [abs(x) for x in bqm_2.quadratic.values()])

In [29]:
for c in chain_strengths:
    sampleset = sampler.sample(bqm_2, annealing_time = 100, num_reads=2000, chain_strength = c*max_val)
    first = sampleset.first
    print(is_sample_feasible(first.sample,N))
    print(sample_to_path(first.sample,N),first.energy, first.chain_break_fraction)

False
[5, 2, 3, 1, 6, 6, 4, 0] 57.0 0.828125
False
[0, 6, 3, 1, 2, 7] 63.0 0.53125
False
[1, 5, 3, 5, 6, 7, 2, 4, 6] 128.0 0.546875
False
[1, 7, 5, 2, 0, 4, 3] 44.0 0.40625
False
[1, 7, 2, 3, 6, 4, 5] 50.0 0.203125


We observe that except chain strength 0.5, we get a feasible solution in all cases. The best solution is obtained when relative chain strength is set to 0.4. Note that every time you run the experiment, you may get a different result. 